In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
import sys
print(sys.path)

## Overlaid Histograms

In [ ]:
temps = Table.read_table('temperatures.csv')
temps.show(3)

In [ ]:
temps.hist('TMIN', unit='degree')

In [ ]:
temps.select('TMAX', 'TMIN').relabeled('TMAX', 'High Temperature').relabeled('TMIN', 'Low Temperature').hist(unit='degree')

## Prediction

In [ ]:
families = Table.read_table('family_heights.csv')
families

In [ ]:
families.where('m/f', 'male').select(1, 2, 3).hist(unit='inch')

In [ ]:
parent_avgs = (families.column('father') + families.column('mother'))/2

In [ ]:
heights = Table().with_columns(
    'Parent Average', parent_avgs,
    'Child', families.column('child'),
    'M/F', families.column('m/f')
)
heights

In [ ]:
heights.scatter(0, 1)

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2);

In [ ]:
nearby = heights.where('Parent Average', are.between(67.5, 68.5))
nearby_mean = np.average(nearby.column('Child'))
nearby_mean

In [ ]:
def predict(h):
    nearby = heights.where('Parent Average', are.between(h - 0.5, h + 0.5))
    nearby_mean = np.average(nearby.column('Child'))
    return nearby_mean 

In [ ]:
predict(68)

In [ ]:
predict(70)

In [ ]:
predict(73)

In [ ]:
predict(62)

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([69.5, 69.5], [50, 85], color='red', lw=2)
plots.plot([70.5, 70.5], [50, 85], color='red', lw=2)
plots.scatter(70, predict(70), color='red', s=50);

In [ ]:
predicted_heights = heights.apply(predict, 'Parent Average')

In [ ]:
heights = heights.with_column('Prediction', predicted_heights)

In [ ]:
heights.select('Parent Average', 'Child', 'Prediction').scatter('Parent Average')

## Prediction Accuracy ##

In [ ]:
def difference(x, y):
    return x - y

In [ ]:
pred_errs = heights.apply(difference, 'Prediction', 'Child')
heights = heights.with_column('errors', pred_errs)
heights

In [ ]:
heights.hist('errors')

# Discussion Question

In [ ]:
def predict_smarter(h, s):
    nearby = heights.where('Parent Average', are.between(h - 1/2, h + 1/2))
    nearby_same_sex = ...
    return ...

In [ ]:
predict_smarter(68, 'female')

In [ ]:
predict_smarter(68, 'male')

In [ ]:
smarter_predicted_heights = heights.apply(predict_smarter, 'Parent Average', 'M/F')
heights = heights.with_column('Smarter Prediction', smarter_predicted_heights)

In [ ]:
smarter_pred_errs = heights.apply(difference, 'Child', 'Smarter Prediction')
heights = heights.with_column('Smarter Errors', smarter_pred_errs)

In [ ]:
heights.hist('Smarter Errors', group='M/F')